# Imports

In [217]:
import requests
import json
import ee
import geemap
import folium
import os
import pandas as pd
import glob
import time
import math
import numpy as np
from datetime import datetime

import modules.agstack_to_gee as agstack_to_gee
import modules.area_stats as area_stats
import modules.tidy_tables as tidy_tables

print ("imports complete")

imports complete


In [218]:
ee.Initialize()

### Parameters

Web app deployed URLs

In [219]:
asset_registry_base = "https://api-ar.agstack.org"
user_registry_base = "https://user-registry.agstack.org"

Add your email and password to register with Agstack and test out the APIs

In [220]:
from parameters.config_asr_credentials import *

Input and output: naming file paths and columns

In [221]:
from parameters.config_output_naming import *

Local deforestation alerts and local buffer parameters

In [222]:
from parameters.config_radd_alerts import *

Optional parameters for defining random polygons

In [223]:
from parameters.config_random_features  import *

### 1. Grab datasets


In [224]:
st = time.time()
if debug: print ("starting image prep...")

from gee.glad_gfc import * #need to create a template
from gee.glad_lulc import *
from gee.glad_pht import *
from gee.birdlife_kbas_biodiversity import *
from gee.creaf_descals_palm import *
from gee.esri_lulc import *
from gee.eth_kalischek_cocoa import *
from gee.fao_gaul_countries import *
from gee.fdap_palm import *
from gee.jaxa_forest import *
from gee.jrc_tmf import *
from gee.wcmc_oecms_protection import *
from gee.wcmc_wdpa_protection import *
from gee.wur_radd_alerts import *

# get the execution time
elapsed_time = time.time() - st

if debug: print ("image prep complete")
if debug: print ('Total execution time:', elapsed_time, 'seconds')

starting image prep...
image prep complete
Total execution time: 0.0009133815765380859 seconds


#### Set properties for which images need special treatment

In [225]:
#deforestation alerts
# set property so run stats for a buffer around site; 
# and show presence only as output 
latest_radd_alert_confirmed_recent_area_hectares = latest_radd_alert_confirmed_recent_area_hectares.setMulti(
    {"alerts_buffer":1,"presence_only_flag":1})

#important sites: 1) protected areas and 2) KBAs (likely future protectred areas) 
# show presence only as output 
protected_areas_WDPA_area_hectares = protected_areas_WDPA_area_hectares.set("presence_only_flag",1)

OECM_2023_area_hectares = OECM_2023_area_hectares.set("presence_only_flag",1)

kba_2023_area_hectares = kba_2023_area_hectares.set("presence_only_flag",1)



#### Create dictionary of images and image names
- prep for reduceRegions statistics so name of datasets/image is added to area stats
- sets "system:index" property of each image
- result is an image collection with 

In [226]:
image_names_dict0 = {
                "GFC_Tree_Cover_2020":gfc_treecover_2020_area_hectares,
                "ESRI_Trees_2020":esri_trees_2020_area_hectares,
                "JAXA_Forest_non_forest_2020":JAXA_forestNonForest_2020_area_hectares,
                "GLAD_LULC_Stable_Tree_2020":glad_stable_tree_2020_area_hectares,
                "TMF_undisturbed_forest_2020":JRC_TMF_undisturbed_2020_area_hectares,
                "Primary_Humid_Tropical_Forest_2020": primary_HT_forests_2020_area_hectares,
                "TMF_disturbed_forest_2020": JRC_TMF_disturbed_2020_area_hectares,
                "Local_RADD_alerts":latest_radd_alert_confirmed_recent_area_hectares,
                "TMF_plantation":JRC_TMF_plantation_area_hectares,
                "Oil_palm_Descals": oil_palm_descals_binary_area_hectares,
                "FDaP_palm_plantations": FDaP_palm_2020_model_area_hectares,
                "Cocoa_plantations_Kalischek": cocoa_map_kalischek_threshold_area_hectares,
                "Protected_area":protected_areas_WDPA_area_hectares,
                "Other_Effective_area_based_Conservation_Measure":OECM_2023_area_hectares,
                "Key_Biodiversity_Area": kba_2023_area_hectares,
                "GAUL_boundaries_adm0_code_reproj":GAUL_boundaries_adm0_code_reproj
              }


In [227]:
#create empty dictionary to be populated
image_names_dict={} 
    
#set image names ("system:index") from keys in dictionary, and store as new one
for i in range(len(image_names_dict0)):
    dataset_name = (list(image_names_dict0.keys())[i]) #get dataset name
    image = (list(image_names_dict0.values())[i]) #get image
    updated_image=image.set("system:index",dataset_name) #set dataset name as image name i.e., "system:index"
    instance={dataset_name:updated_image} #combine
    image_names_dict.update(instance) #update into new dictionary

del image_names_dict0 # remove old dictionary

#make into a new image collection
images_IC = ee.ImageCollection(list(image_names_dict.values()))
                      
##checks
if debug: print ("number of images: ",len(image_names_dict))

images_IC

number of images:  16


### 3. Fetch some fields (public)

#### Transform geometries into a feature collection

In [228]:
CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
'97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

all_geo_ids= CIV_ids+GHA_ids+IDN_ids

if debug: print (all_geo_ids)

['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918', 'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f', 'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3', 'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f', '88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c', '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb', 'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7', '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45', '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']


#### Asset registry: start session

NB this is timing out so skipping section - seems to work without somehow. Maybe an open connection already...

In [229]:
## using session to store cookies that are persistent
session = requests.session()
session.headers = headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
req_body = {'email': email, 'password': password}
res = session.post(user_registry_base, json=req_body)
if debug: print (session.cookies)
if debug: print (res.status_code)


<RequestsCookieJar[]>
500


#### Fetch features

In [230]:
#is there a list of geo_ids
if isinstance(all_geo_ids, list):
    multiple_inputs=True
elif isinstance(all_geo_ids, str):
    multiple_inputs=False
else:
    print ("Input must be a single string or list of strings")

#if list of geo ids use loop over them and make a feature collection

if multiple_inputs==True:
    roi = agstack_to_gee.geo_id_list_to_feature_collection(all_geo_ids,geo_id_column, session,asset_registry_base)    
    if debug: print ("Count of geo ids in list: ", len(all_geo_ids))
    # if debug: print ("Count of features in FeatureCollection: ", roi.size().getInfo())
elif multiple_inputs == False: 
    roi = ee.FeatureCollection(agstack_to_gee.geo_id_to_feature(all_geo_ids,geo_id_column, session,asset_registry_base))
    if debug: print ("Geo id input: ", all_geo_ids)
    # if debug: print ("Geo id associated with new feature: ", roi.get(geo_id_column).getInfo())    
else: 
    print("no ee.Object created: check input format")

# if debug: print ("Count of geo ids in list: ", len(all_geo_ids))
if debug: print ("Count of features in FeatureCollection: ", roi.size().getInfo())
    
# if (ee.Algorithms.ObjectType(roi).getInfo() == "Feature"):
# elif ee.Algorithms.ObjectType(roi).getInfo == "FeatureCollection":
    
#true
#checks 

Count of geo ids in list:  11
Count of features in FeatureCollection:  11


##### Alternative feature collection: create random polygons
select random points inside administrative boundaries and buffer them 

In [238]:
# def create_random_points_in_polys(feature): #to tidy
#     """ creates random points within either a polygon or a feature collection NB relies upon some globals being set currently"""
#     return ee.FeatureCollection.randomPoints(region = feature.geometry(max_error), points = number_of_points, seed=seed, maxError=10)

# admin_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2").filter(
#     ee.Filter.inList("ADM0_NAME",["Indonesia","Malaysia","Ghana"]))##.filter(ee.Filter.lt("Shape_Area",1000))

# random_collection = admin_boundaries.randomColumn(seed = seed).sort('random').limit(number_of_boundaries)

# if points_per_admin_boundary:
#     random_points = random_collection.map(create_random_points_in_polys).flatten()
# else:
#     random_points = create_random_points_in_polys(random_collection)

# random_buffers = random_points.map(lambda feature: feature.buffer(buffer_distance_meters,100)) ##buffer by distance in meters

# roi= random_buffers.map(lambda feature: feature.set(geo_id_column,(feature.get("system:index"))))
# # roi= random_buffers.map(set_geo_id_from_system_index) ## add surrogate "geo_id" for formatting

# #checks
# if debug: print ("number of admin regions",random_collection.size().getInfo())
# if debug: print ("number of countries",random_collection.aggregate_array("ADM0_NAME").distinct().getInfo())
# if debug: print ("number of buffers created",random_buffers.size().getInfo())
#     # geemap.ee_to_pandas(roi)###create random points and buffer them


In [239]:
# ## checks
# Map=geemap.Map()
# Map.addLayer(roi, "", 'roi', 1, 1)

# Map

#### Feature prep
- Create additional buffer zones for deforestation risk 
- Add area property to feature(s) 
- Select only columns of interest

In [240]:
roi = area_stats.add_area_hectares_property_to_feature_collection(roi,geometry_area_column)

roi  = roi.select([geometry_area_column,geo_id_column]) ##select only fields of interest

roi_alerts_buffer = roi.map(lambda feature: 
        feature.buffer(local_alerts_buffer_radius,max_error_alert_buff))

geemap.ee_to_pandas(roi_alerts_buffer)
geemap.ee_to_pandas(roi)


,Shape_area_hectares,Geo_id
0,8.316153,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...
1,1.990210,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...
2,3.815523,b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0...
3,3.628190,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...
4,1.947148,88bec54ad04804f5b1fafbc131266640a129be2840fa67...
5,4.171951,ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e45485042...
6,16.676077,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...
7,31.353357,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...
8,1.973565,1a41a309ae2387f36a604c9a6c81887e64357a7f61d228...
9,12.782163,1a4472dc40700ef33f931863f58d444f243d6441861667...


#### Compute statistics

Calculating zonal statistics for continuous data (e.g tree cover) within polygon(s)

##### Step 1 Mapping over image collection with reduce regions: creates long format raw stats

In [243]:
# get the start time
st = time.time()
if debug: print ("processing stats...")


## reducer choice for zonal statistics
reducer_choice = ee.Reducer.sum().combine(  #main stats based on area of pixel
  reducer2=ee.Reducer.count(),sharedInputs=True).combine(reducer2=ee.Reducer.mode(), sharedInputs=True) ##used for country allocation (majority pixel count on country code raster)


#get stats for roi (except alerts)
fc_stats_combined = area_stats.reduceStatsIC(roi,
                                  images_IC.filter(ee.Filter.neq("alerts_buffer",1)),
                                  reducer_choice)# all but alerts
#get stat for buffer (alerts only)
fc_stats_combined_buffer = area_stats.reduceStatsIC(roi_alerts_buffer,
                                         images_IC.filter(ee.Filter.eq("alerts_buffer",1)),
                                         reducer_choice) #alerts only

#combine stats from roi and buffer
fc_stats_combined_all = fc_stats_combined.merge(fc_stats_combined_buffer) # combining alerts with others into one feature collection

# convert to Pandas Dataframe
df_combined = geemap.ee_to_pandas(fc_stats_combined_all) # limit of 5000 (unlikely to need more but i have code for it if needed)

# export dataframe to csv
df_combined.to_csv(path_or_buf=out_file_long,header=True,index=False)

# get the execution time
elapsed_time = time.time() - st

if debug: print ('Total execution time:', elapsed_time, 'seconds')


processing stats...
Total execution time: 2.3458592891693115 seconds


##### Step 2: Create lookup tables for country allocation
Approach is based on raster stats and listing the country for a specific geometry based on which has most overlap



Look up table linking country codes to country names (from GAUL feature collection) is stored here: scripts: create_country_lookup.py

Make on-the-fly look up table to link country name to geo id based on raster stats
- uses rasterised GAUL layer with admin codes as pixel values
- for each geo id finds most common value in that geometry (i.e. "mode" statistic)

In [244]:
path_lookup_country_codes_to_names = "parameters/lookup_GAUL_country_codes_to_names.csv" # to put somewhere in parameters

lookup_country_codes_to_names = pd.read_csv(path_lookup_country_codes_to_names)

lookup_geo_id_to_GAUL = df_combined[df_combined["dataset_name"]=="GAUL_boundaries_adm0_code_reproj"]  #get mode satats for GAUL dataset

lookup_geo_id_to_GAUL = lookup_geo_id_to_GAUL[[geo_id_column, 'mode']] # choose only columns needed

lookup_geo_id_to_GAUL["mode"] = lookup_geo_id_to_GAUL["mode"].astype(int) # make sure mode stats are integer (to allow joining)

lookup_geo_id_to_GAUL.rename(columns={"mode":"ADM0_CODE"},inplace=True) # change names for a clean join 

lookup_geo_id_to_GAUL_country_names = lookup_geo_id_to_GAUL.merge(lookup_country_codes_to_names,on="ADM0_CODE",how="inner") # join geo id to the GAUL_lookup_table countaining "Country_names" 

if debug: lookup_geo_id_to_GAUL_country_names

##### Step 3 Reformat results table
- long to wide
- convert to proportions 
- set presence only flags
- add in country names (using lookup tables) to the final results

In [245]:
#add proprtion column
df_combined["percentage"] = (df_combined["sum"]/df_combined[geometry_area_column])*100

# geometry_area_lookup = df_combined[geometry_area_column,geo_id_column]

def tidy_dataframe_after_pivot (df):
    """Tidying dataframe after long-to-wide reformatting, incl. removes unwanted levels, column names"""
    # df.columns = df.columns.droplevel(0) #remove sum
    df.columns = df.columns.get_level_values(1)
    df.columns.name = None               #remove "dataset_name" label
    df = df.reset_index()    #index to columns
    return df

#convert to wide format (one row per geo_id)
df_wide_format = df_combined.pivot_table(index=[geo_id_column,geometry_area_column],columns=['dataset_name'],values=['percentage'])

# # #tidy unwanted headers etc
tidy_tables.tidy_dataframe_after_pivot(df_wide_format) #runs in place so no need to assign

df_wide_format.columns = df_wide_format.columns.get_level_values(0)

# #list images with with presence_only_flag property 
flag_list = images_IC.filter(ee.Filter.eq("presence_only_flag",1)).aggregate_array("system:index").getInfo()

# convert positive results values to "True" for specific columns
for column in flag_list: df_wide_format[column]=np.where(df_wide_format[column]>0,"True","-")

# # tidy output - decimal places
columns_list = df_wide_format.columns.values.tolist()
if debug: print (columns_list)
non_flag_columns_list = [x for x in columns_list if x not in flag_list]

for column in flag_list: df_wide_format[non_flag_columns_list]=df_wide_format[non_flag_columns_list].round(decimals=0, out=None).astype(int)

df_wide_format=df_wide_format.reset_index()

# #joins country name based on majority overlap with country 
df_wide_format = df_wide_format.merge(lookup_geo_id_to_GAUL_country_names,on=geo_id_column).drop("ADM0_CODE",axis=1).drop("GAUL_boundaries_adm0_code_reproj",axis=1)




['Cocoa_plantations_Kalischek', 'ESRI_Trees_2020', 'FDaP_palm_plantations', 'GAUL_boundaries_adm0_code_reproj', 'GFC_Tree_Cover_2020', 'GLAD_LULC_Stable_Tree_2020', 'JAXA_Forest_non_forest_2020', 'Key_Biodiversity_Area', 'Local_RADD_alerts', 'Oil_palm_Descals', 'Other_Effective_area_based_Conservation_Measure', 'Primary_Humid_Tropical_Forest_2020', 'Protected_area', 'TMF_disturbed_forest_2020', 'TMF_plantation', 'TMF_undisturbed_forest_2020']


##### Step 4 Further reformatting and exporting
- reorder columns
- remove underscores in column titles
- export to csv

In [246]:
##tidy 

# reorder columns using list 
df_wide_format[geometry_area_column]=df_wide_format[geometry_area_column].round(decimals=2, out=None)

column_order_list = list(image_names_dict.keys()) # get list of input datasets

column_order_list.insert(0,geo_id_column) # add in the "geo_id" column into to datasets list

column_order_list.insert(1,geometry_area_column)# add in to list the geometry area column

column_order_list.remove("GAUL_boundaries_adm0_code_reproj") # remove old column with "mode" values (not now relevant as have country names)

column_order_list.insert(2,"Country")# add in to list the new column with country names

df_wide_format= df_wide_format.reindex(columns=column_order_list) # reorder by list

df_wide_format["Country"]=np.where(df_wide_format["Country"]=="C�te d'Ivoire","Côte d'Ivoire",df_wide_format["Country"])# TEMP fix on characters (encoding issues)
df_wide_format["Country"]=np.where(df_wide_format["Country"]=="R�union","Réunion",df_wide_format["Country"])# TEMP fix on characters (encoding issues)

# remove underscores in columns
df_wide_format.columns = df_wide_format.columns.str.replace('_', ' ')

# #export wide format csv
df_wide_format.to_csv(path_or_buf=out_file_wide,header=True)

# if debug: print ("output csv: ", out_file_wide)

#checks
if debug: flag_list
# if debug: print (columns_list)
df_wide_format

,Geo id,Shape area hectares,Country,GFC Tree Cover 2020,ESRI Trees 2020,JAXA Forest non forest 2020,GLAD LULC Stable Tree 2020,TMF undisturbed forest 2020,Primary Humid Tropical Forest 2020,TMF disturbed forest 2020,Local RADD alerts,TMF plantation,Oil palm Descals,FDaP palm plantations,Cocoa plantations Kalischek,Protected area,Other Effective area based Conservation Measure,Key Biodiversity Area
0,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.32,Côte d'Ivoire,62,100,95,64,0,0,10,True,0,0,0,0,True,-,True
1,1a41a309ae2387f36a604c9a6c81887e64357a7f61d228...,1.97,Indonesia,0,100,85,99,0,0,13,-,0,0,4,0,-,-,-
2,1a4472dc40700ef33f931863f58d444f243d6441861667...,12.78,Indonesia,83,100,100,99,60,0,39,-,0,0,68,0,-,-,-
3,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.95,Ghana,20,100,100,88,0,0,34,-,0,0,0,0,-,-,-
4,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.98,Indonesia,53,100,96,98,31,0,67,-,0,0,51,0,-,-,-
5,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.68,Ghana,100,100,100,89,95,87,5,True,0,0,0,0,True,-,-
6,b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0...,3.82,Côte d'Ivoire,70,100,99,90,0,0,74,-,0,0,0,0,-,-,-
7,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...,1.99,Côte d'Ivoire,78,100,73,55,0,0,23,True,0,0,0,73,True,-,-
8,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.35,Indonesia,0,6,5,83,0,0,0,-,100,98,82,0,-,-,-
9,ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e45485042...,4.17,Ghana,35,100,99,63,5,0,83,-,0,0,0,0,-,-,-


#### Display layers

##### Loop through image collection and loads layers to be added to the map


Map = geemap.Map()
visParams =  {'min': 0,'max': 1,'palette':['White','Green']}

for i in range(images_IC.size().getInfo()):
    
    imageNew = ee.Image(images_IC.toList(100,0).get(i))
    
    dataset_name = (list(image_names_dict.keys())[i])
    
    if debug: print ("adding image",i,"-",dataset_name)
    Map.addLayer(imageNew.gt(0).unmask(),visParams,dataset_name,0,1)
    
    
Map.addLayer(roi,{},'roi ',1,1)
# Map.addLayer(roi_alerts_buffer,{},'roi buffer zone')

if debug: print ("All layers added")    

##### Show on map and zoom to a specific feature based on index number in feature collection
 Layers visibility off by default - toggle on in top right corner

In [249]:

# number/index from list of ROI features - the selected feature is shown on the map. e.g., choose 0 for first in the list 
feature_to_centre_on = 1

#choose how close to zoom to chosen polygon (1-24, where 24 is fully zoomed in) 
zoom_level = 16 

single_feature_id = roi.aggregate_array(geo_id_column).get(feature_to_centre_on).getInfo()
if debug: print (geo_id_column,single_feature_id)
single_feature = ee.Feature(roi.filter(ee.Filter.eq(geo_id_column,single_feature_id)).first())

Map.centerObject(single_feature,zoom_level)
    
Map
# # single_feature


Geo_id b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f


Map(bottom=8124056.0, center=[5.673810322182612, -4.086848133220072], controls=(WidgetControl(options=['positi…

#### Optional: export image collection to asset

- code for exporting to a collection, which would improve load times NB code needs tidying; "mode" stats note working thought fdor country allocation...

In [211]:
export_image_collection_to_asset =False # NB shift to parameters if code is kept in this script


if export_image_collection_to_asset:

    targetImageCollId = "users/andyarnell10/fdap_dpi/imageCol_trial"

    createNewImageColl = True # if true then code will add outputImageColl if it doesn't exist already. Code at end of cell/section.

    skipExportIfAssetExists = True # if image with the same name exists avoid exporting

    exportRegion = admin_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2").filter(
        ee.Filter.inList("ADM0_NAME",["Ghana"])).geometry() # for once country only


    if createNewImageColl == True:
        try:
            getAssetInfo = ee.data.getAsset(targetImageCollId)
            if debug: print ("Target image collection exists: ",targetImageCollId)
        except:
            ee.data.createAsset({'type': 'ImageCollection'}, targetImageCollId)#make a new image collection
            print ("New (empty) image collection created: ",targetImageCollId)
            skipExportIfAssetExists = True# as it sounds like. Saves possibility of lots of red errors in Tasks list in code editor

        def imageNames (imageCollection):##list existing images in collection (if any)
            return imageCollection.aggregate_array("system:id").getInfo()

    imageCollectionImageList = (imageNames(ee.ImageCollection(targetImageCollId)))


    for i in range(images_IC.size().getInfo()):

        imageNew = ee.Image(images_IC.toList(100,0).get(i))

        dataset_name = (list(image_names_dict.keys())[i])

        output_scale = imageNew.get("scale").getInfo()

        out_name = targetImageCollId+"/"+dataset_name

        task = ee.batch.Export.image.toAsset(image= imageNew,\
                                         description= dataset_name,\
                                         assetId=out_name,\
                                         scale= output_scale,\
                                         maxPixels=1e13,\
                                         region=exportRegion)

        if ((skipExportIfAssetExists==True) and (out_name in imageCollectionImageList)):
            if debug: print ("testing - not exporting NB asset exists")
        else:
            task.start()###code out if testing and dont want to export assets

            if debug: print ("exporting image: "+ out_name)


if debug: print ("finished")

finished


### Logout (protected)

In [ ]:
# res = session.get(asset_registry_base + "/logout")
# if debug: print (res.json())
# res = session.get(user_registry_base + "/logout", cookies=session.cookies)
# session.headers.clear()

### Checking if Logged out correctly

In [ ]:
# # Confirming the logout from Asset Registry by requesting a Protected route
# req_body = {
#     "latitude": 31.47704430446457,
#     "longitude": 74.37510786779589
# }
# res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
# if debug: print (res.json())

### Get all Domains (public)

In [ ]:
# # Fetching all the domains from the User Registry
# res = session.get(asset_registry_base + "/domains")
# if debug: print (res.json())